# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

geoapify_key = "f76b71d9aa0c4c4d8e5b01cff5e06c63"

print(geoapify_key)

f76b71d9aa0c4c4d8e5b01cff5e06c63


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lucapa,-8.4192,20.7447,290.76,96,99,1.05,AO,1674523668
1,1,padang,-0.9492,100.3543,297.18,89,100,1.78,ID,1674523505
2,2,punta arenas,-53.1500,-70.9167,284.21,50,0,11.83,CL,1674523489
3,3,kaitangata,-46.2817,169.8464,288.23,56,49,4.21,NZ,1674523634
4,4,bluff,-46.6000,168.3333,286.73,58,13,4.43,NZ,1674523566


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
map_plot=city_data_df.hvplot.points(
    "Lng","Lat",
    tiles = "OSM",
    frame_width=500,
    frame_height=500,
    size="Humidity",
    scale=0.7)

# Display the map

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data
# Zero cloudiness
cleaned_df = city_data_df.loc[(city_data_df['Max Temp']>27)&(city_data_df['Max Temp']>21)&
                            (city_data_df["Wind Speed"]<4.5)&(city_data_df["Cloudiness"]==0)]
cleaned_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,puerto escondido,15.8500,-97.0667,299.61,69,0,2.06,MX,1674523412
41,41,clyde river,70.4692,-68.5914,237.28,68,0,1.54,CA,1674523679
69,69,chuy,-33.6971,-53.4616,291.60,71,0,2.27,UY,1674523599
75,75,mar del plata,-38.0023,-57.5575,293.20,76,0,4.47,AR,1674523587
84,84,elizabethtown,37.6940,-85.8591,273.20,87,0,0.00,US,1674523472


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

hotel_df=cleaned_df.loc[:,["City_ID", "City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"]=""
# Display sample data
# YOUR CODE HERE

hotel_df.reset_index()

,index,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
0,18,18,puerto escondido,MX,15.8500,-97.0667,69,
1,41,41,clyde river,CA,70.4692,-68.5914,68,
2,69,69,chuy,UY,-33.6971,-53.4616,71,
3,75,75,mar del plata,AR,-38.0023,-57.5575,76,
4,84,84,elizabethtown,US,37.6940,-85.8591,87,
...,...,...,...,...,...,...,...,...
61,535,535,kumluca,TR,36.3703,30.2869,53,
62,545,545,hurghada,EG,27.2574,33.8129,55,
63,549,549,hami,CN,42.8000,93.4500,38,
64,566,566,iberia,US,29.9502,-91.7507,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":"20",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city_lat=hotel_df.loc[index,"Lat"]
    city_lng=hotel_df.loc[index,"Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_lng},{city_lat},{radius}"
    params["bias"] = f"proximity:{city_lng},{city_lat}"
       
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto escondido - nearest hotel: Hotel Barlovento
clyde river - nearest hotel: No hotel found
chuy - nearest hotel: Alerces
mar del plata - nearest hotel: Nuevo Ostende
elizabethtown - nearest hotel: No hotel found
coos bay - nearest hotel: No hotel found
afsin - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
abnub - nearest hotel: University Guest House
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
tucuman - nearest hotel: Suites San Luis
erzin - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
rocha - nearest hotel: Hotel Municipal
north bend - nearest hotel: The Mill Casino Hotel
acapulco - nearest hotel: Hotel del Valle
esperance - nearest hotel: Hospitality
saldanha - nearest hotel: Hoedjesbaai
bayir - nearest hotel: No hotel found
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
araouane - nearest hotel: No hotel found
diego de almagro - nea

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
18,18,puerto escondido,MX,15.8500,-97.0667,69,Hotel Barlovento
41,41,clyde river,CA,70.4692,-68.5914,68,No hotel found
69,69,chuy,UY,-33.6971,-53.4616,71,Alerces
75,75,mar del plata,AR,-38.0023,-57.5575,76,Nuevo Ostende
84,84,elizabethtown,US,37.6940,-85.8591,87,No hotel found
...,...,...,...,...,...,...,...
535,535,kumluca,TR,36.3703,30.2869,53,Uyguluma Oteli
545,545,hurghada,EG,27.2574,33.8129,55,Sea Horse Hotel
549,549,hami,CN,42.8000,93.4500,38,Хами
566,566,iberia,US,29.9502,-91.7507,62,Inn of Iberia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

map_plot_hotel=hotel_df.hvplot.points(
    "Lng","Lat",
    tiles = "OSM",
    frame_width=750,
    frame_height=500,
    size="Humidity",
    scale=0.9,
    hover_cols=["Hotel Name","Country"]
    )
# Display the map
# YOUR CODE HERE
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)